<a href="https://colab.research.google.com/github/GrigoreGeorgeAlexandru/Colab-projects/blob/main/Laborator6_A_Linear_Regression_Example_stud.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyspark


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=0e570a2c1c12e417b9d1e4188b3baba32a03d7c19ba771d0465fbfef7a874ed8
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


# Exemplu regresie liniară

Urmărim pașii exemplelor din documentația oficială, cu scopul de a înțelege și apoi aplica metodele pentru rezolvarea altor probleme.
<br>
https://spark.apache.org/docs/latest/ml-classification-regression.html#linear-regression
<br>
https://spark.apache.org/docs/latest/api/java/index.html?org/apache/spark/ml/regression/LinearRegression.html

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from pyspark.sql import SparkSession

In [ ]:
spark = SparkSession.builder.appName('lr_example').getOrCreate()

In [ ]:
from pyspark.ml.regression import LinearRegression

In [ ]:
# Încărcarea datelor de antrenare
training = spark.read.format("libsvm").option("numFeatures","10").load("/content/drive/MyDrive/Data/sample_linear_regression_data.txt")

Observatie: "libsvm" este un format de date, structurat astfel:
`<label> <index1>:<value1> <index2>:<value2> `
<br>
Observatie: Dataset-uri libsvm: https://www.csie.ntu.edu.tw/~cjlin/libsvmtools/datasets/

In [ ]:
training.show(truncate=False)

+-------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|label              |features                                                                                                                                                                                                                                |
+-------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|-9.490009878824548 |(10,[0,1,2,3,4,5,6,7,8,9],[0.4551273600657362,0.36644694351969087,-0.38256108933468047,-0.4458430198517267,0.33109790358914726,0.8067445293443565,-0.2624341731773887,-0.44850386111659524,-0.07269284838169332,0.5658

Acesta este formatul așteptat de Spark, alcătuit din 2 coloane (label, features).

Coloana "label" trebuie să aibă o valoare numerică, fie o valoare numerică de regresie, fie o valoare numerică ce corespunde unui grup dintr-o clasificare.

Coloana "features" conține un vector având valorile caracteristicilor corespunzătoare acelui rând. De obicei, se ajunge la combinarea diferitelor coloane cu caracteristici într-o singură coloană "features" cu ajutorul transformărilor.

In [ ]:
# Acestea sunt valorile implicite pentru featuresCol, labelCol, predictionCol
lr = LinearRegression(featuresCol='features', labelCol='label', predictionCol='prediction')

# Putem utiliza și alți parametri - de exemplu,
#pentru regularizare (vezi https://static1.squarespace.com/static/5ff2adbe3fe4fe33db902812/t/6062a083acbfe82c7195b27d/1617076404560/ISLR%2BSeventh%2BPrinting.pdf)

In [ ]:
# Antrenați modelul cu datele "training"
lrModel = lr.fit(training)

In [ ]:
# Afișați coeficienții și intercepția pentru regresia liniară
print("Coefficients: {}".format(str(lrModel.coefficients)))
print('\n')
print("Intercept:{}".format(str(lrModel.intercept)))

Coefficients: [0.0073350710225801715,0.8313757584337543,-0.8095307954684084,2.441191686884721,0.5191713795290002,1.1534591903547016,-0.2989124112808717,-0.5128514186201779,-0.619712827067017,0.695615180432293]


Intercept:0.14228558260358093


Observație: Există un atribut (`summary`) ce conține mai multe informații.

In [ ]:
# Sumarizarea modelului pentru setul de training și afișarea unor metrici
trainingSummary = lrModel.summary

Analizați documentația și afișați câteva exemple de informații (ce reprezintă fiecare?):

In [ ]:
trainingSummary.residuals.show()
print("RMSE: {}".format(trainingSummary.rootMeanSquaredError))
print("r2: {}".format(trainingSummary.r2))

+-------------------+
|          residuals|
+-------------------+
|-11.011130022096554|
| 0.9236590911176535|
|-4.5957401897776675|
|  -20.4201774575836|
|-10.339160314788181|
|-5.9552091439610555|
|-10.726906349283922|
| 2.1228071931912336|
|  4.077122222293811|
|-17.316168071241652|
| -4.593044343959059|
|  6.380476690746936|
| 11.320566035059846|
|-20.721971774534094|
| -2.736692773777401|
| -16.66886934252847|
|  8.242186378876315|
|-1.3723486332690233|
|-0.7060332131264666|
|-1.1591135969994064|
+-------------------+
only showing top 20 rows

RMSE: 10.16309157133015
r2: 0.027839179518600154


## Diviziuni Train/Test

Până în acest punct nu am separat setul de date într-un subset de training și unul de test, ci am antrenat pe toate datele - lucru pe care, în general, dorim să îl evităm.
De reținut: nu se poate face o evaluare corectă a unui model analizând performanța lui pe datele pe care a fost antrenat!

În Spark DataFrame există metoda `randomSplit` pentru divizarea aleatoare a datelor. Utilizăm această metodă:

In [ ]:
all_data = spark.read.format("libsvm").option("numFeatures","10").load("/content/drive/MyDrive/Data/sample_linear_regression_data.txt")

In [ ]:
# Procentele de divizare pentru seturile de training/test sunt transmise ca listă.
# În general, sunt utilizate procentele 70%/30% sau 60%/40%, în funcție de volumul de date disponibil și de cât de echilibrate sunt acestea.
train_data,test_data = all_data.randomSplit([0.7,0.3])

In [ ]:
train_data.show()

+-------------------+--------------------+
|              label|            features|
+-------------------+--------------------+
|-28.571478869743427|(10,[0,1,2,3,4,5,...|
|-28.046018037776633|(10,[0,1,2,3,4,5,...|
|-26.805483428483072|(10,[0,1,2,3,4,5,...|
|-23.487440120936512|(10,[0,1,2,3,4,5,...|
|-22.949825936196074|(10,[0,1,2,3,4,5,...|
|-22.837460416919342|(10,[0,1,2,3,4,5,...|
|-21.432387764165806|(10,[0,1,2,3,4,5,...|
|-20.057482615789212|(10,[0,1,2,3,4,5,...|
|-19.884560774273424|(10,[0,1,2,3,4,5,...|
|-19.872991038068406|(10,[0,1,2,3,4,5,...|
| -19.66731861537172|(10,[0,1,2,3,4,5,...|
| -19.16829262296376|(10,[0,1,2,3,4,5,...|
|-18.845922472898582|(10,[0,1,2,3,4,5,...|
| -18.27521356600463|(10,[0,1,2,3,4,5,...|
|-17.803626188664516|(10,[0,1,2,3,4,5,...|
|-17.494200356883344|(10,[0,1,2,3,4,5,...|
|-17.428674570939506|(10,[0,1,2,3,4,5,...|
|-17.065399625876015|(10,[0,1,2,3,4,5,...|
| -16.71909683360509|(10,[0,1,2,3,4,5,...|
|-16.692207021311106|(10,[0,1,2,3,4,5,...|
+----------

In [ ]:
test_data.show()

+-------------------+--------------------+
|              label|            features|
+-------------------+--------------------+
|-26.736207182601724|(10,[0,1,2,3,4,5,...|
| -23.51088409032297|(10,[0,1,2,3,4,5,...|
|-20.212077258958672|(10,[0,1,2,3,4,5,...|
|-19.782762789614537|(10,[0,1,2,3,4,5,...|
|-19.402336030214553|(10,[0,1,2,3,4,5,...|
| -17.32672073267595|(10,[0,1,2,3,4,5,...|
|-17.026492264209548|(10,[0,1,2,3,4,5,...|
| -16.26143027545273|(10,[0,1,2,3,4,5,...|
| -15.86200932757056|(10,[0,1,2,3,4,5,...|
|-15.056482974542433|(10,[0,1,2,3,4,5,...|
|-13.867087895158768|(10,[0,1,2,3,4,5,...|
|-13.772441561702871|(10,[0,1,2,3,4,5,...|
|-13.039928064104615|(10,[0,1,2,3,4,5,...|
| -12.92222310337042|(10,[0,1,2,3,4,5,...|
|-12.500773785355054|(10,[0,1,2,3,4,5,...|
| -11.87816749996684|(10,[0,1,2,3,4,5,...|
|-11.640549677888826|(10,[0,1,2,3,4,5,...|
| -10.29566593602992|(10,[0,1,2,3,4,5,...|
|-10.041353112580456|(10,[0,1,2,3,4,5,...|
| -9.754451457291598|(10,[0,1,2,3,4,5,...|
+----------

In [ ]:
##Datele de test, fără etichete
unlabeled_data = test_data.select('features')

In [ ]:
unlabeled_data.show()

+--------------------+
|            features|
+--------------------+
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
+--------------------+
only showing top 20 rows



Vom antrena doar pe baza setului train_data

In [ ]:
#Antrenăm doar pe train_data
correct_model = lr.fit(train_data)

Putem obține un obiect "summary" folosind metoda "evaluate":

In [ ]:
test_results = correct_model.evaluate(test_data)

In [ ]:
test_results.residuals.show()
print("RMSE: {}".format(test_results.rootMeanSquaredError))

+-------------------+
|          residuals|
+-------------------+
| -23.18546760225756|
|-21.853757539736687|
| -21.21668501621559|
|-19.472464838700418|
|-18.732379799451987|
|-15.350857319379909|
|-16.751310354257743|
| -18.38274322937227|
|-15.993026063623017|
|-16.083694019024396|
|-12.397354327199158|
| -17.17068279313801|
|-15.080593028842268|
|-17.435994658927537|
|-14.884416229792873|
|-13.637601226722339|
| -7.038494977525212|
| -8.876401617807634|
| -7.121041726033303|
| -8.149659201052156|
+-------------------+
only showing top 20 rows

RMSE: 9.950991558199124


Dorim să testăm modelul pe datele neetichetate - în primul rând, acesta este scopul construirii modelului.
Putem realiza această testare cu ajutorul unei alte metode - `transform()`. Această metodă a fost, de fapt, apelată și în cadrul lui `evaluate()`.

In [ ]:
predictions = correct_model.transform(unlabeled_data)

In [ ]:
predictions.show(),test_data.show()

+--------------------+-------------------+
|            features|         prediction|
+--------------------+-------------------+
|(10,[0,1,2,3,4,5,...|-3.5507395803441644|
|(10,[0,1,2,3,4,5,...|-1.6571265505862853|
|(10,[0,1,2,3,4,5,...|  1.004607757256916|
|(10,[0,1,2,3,4,5,...|-0.3102979509141207|
|(10,[0,1,2,3,4,5,...|-0.6699562307625665|
|(10,[0,1,2,3,4,5,...|  -1.97586341329604|
|(10,[0,1,2,3,4,5,...| -0.275181909951804|
|(10,[0,1,2,3,4,5,...| 2.1213129539195386|
|(10,[0,1,2,3,4,5,...|0.13101673605245767|
|(10,[0,1,2,3,4,5,...| 1.0272110444819638|
|(10,[0,1,2,3,4,5,...|-1.4697335679596097|
|(10,[0,1,2,3,4,5,...|  3.398241231435137|
|(10,[0,1,2,3,4,5,...|  2.040664964737653|
|(10,[0,1,2,3,4,5,...|  4.513771555557116|
|(10,[0,1,2,3,4,5,...| 2.3836424444378177|
|(10,[0,1,2,3,4,5,...| 1.7594337267554976|
|(10,[0,1,2,3,4,5,...| -4.602054700363614|
|(10,[0,1,2,3,4,5,...| -1.419264318222285|
|(10,[0,1,2,3,4,5,...|-2.9203113865471533|
|(10,[0,1,2,3,4,5,...|-1.6047922562394428|
+----------

(None, None)

Ce concluzii se pot trage asupra predicției obținute?